In [ ]:
pip install pypdf langchain torch accelerate bitsandbytes transformers sentence_transformers faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 88.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 19.5 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
#from langchain.llms import CTransformers
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import sys

In [ ]:
DB_FAISS_PATH = "vectorstore/db_faiss"
loader = CSVLoader(file_path="/content/education_df.csv", encoding="utf-8", csv_args={'delimiter': ','})
data = loader.load()
print(data)

[Document(page_content="name: Aditya Ba Baso Patil\nlevel: Bachlor\ninstitution: Indian Institute of Technology, Roorkee\nmajors_subject: B.Tech.\nstatus: {'Completed': 'yes', 'year_or_semester': '3rd Year'}\nyear_of_completion: 2023\nCPI/CGPA/Percentage: 8.116\ndepartment_ranking: \nmajor_subject: ", metadata={'source': '/content/education_df.csv', 'row': 0}), Document(page_content="name: Aditya Ba Baso Patil\nlevel: Intermediate (Class XII)\ninstitution: DAV Public School, New Panvel\nmajors_subject: \nstatus: {'Completed': 'yes', 'year_or_semester': ''}\nyear_of_completion: 2021\nCPI/CGPA/Percentage: 97.40 %\ndepartment_ranking: \nmajor_subject: ", metadata={'source': '/content/education_df.csv', 'row': 1}), Document(page_content="name: Aditya Ba Baso Patil\nlevel: Matriculate (Class X)\ninstitution: New Horizon Public School, New Panvel\nmajors_subject: \nstatus: {'Completed': 'yes', 'year_or_semester': ''}\nyear_of_completion: 2019\nCPI/CGPA/Percentage: 96.00 %\ndepartment_ranking

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)

In [ ]:
print(len(text_chunks))

20


In [ ]:
# Download Sentence Transformers Embedding From Hugging Face
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
docsearch = FAISS.from_documents(text_chunks, embeddings)

In [ ]:
docsearch.save_local(DB_FAISS_PATH)

In [ ]:
pip install ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 24.9 MB/s eta 0:00:00


In [ ]:
from langchain_community.llms import CTransformers
#from ctransformers import AutoModelForCausalLM

# llm = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-7B-Chat-GGML", model_file="llama-2-7b-chat.ggmlv3.q8_0.bin",model_type="llama",
#                     max_new_tokens=512,
#                     temperature=0.1)
config = {'max_new_tokens': 256, 'repetition_penalty': 1.1}
llm = CTransformers(model='TheBloke/Llama-2-7B-Chat-GGML', model_file='llama-2-7b-chat.ggmlv3.q8_0.bin')



Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

llama-2-7b-chat.ggmlv3.q8_0.bin:   0%|          | 0.00/7.16G [00:00<?, ?B/s]

In [ ]:
pip install langchain --upgrade

In [ ]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=docsearch.as_retriever())

In [ ]:
while True:
    chat_history = []
    #query = "What is the value of  GDP per capita of Finland provided in the data?"
    query = input(f"Input Prompt: ")
    if query == 'exit':
        print('Exiting')
        sys.exit()
    if query == '':
        continue
    result = qa({"question":query, "chat_history":chat_history})
    print("Response: ", result['answer'])

Input Prompt: who has phone number?


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


KeyboardInterrupt: 